In [1]:
%cd ..

/home/quasar/GitHub/TradeForecast


/home/quasar/GitHub/TradeForecast/.venv/lib/python3.12/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [2]:
from tradeforecast import Scrapper

ticker = 'AAPL'
scrapper = Scrapper(ticker)

exported_fnames_dict = scrapper.export_historic_data()
exported_fnames_dict

[*********************100%***********************]  1 of 1 completed


{'AAPL': 'AAPL_1d_max_(None-None).csv'}

In [3]:
from tradeforecast.augmentation import DataEntryPoint, RNNDataset, train_test_split

data_entry = DataEntryPoint(exported_fnames_dict['AAPL'])

lf = data_entry.data.drop_nulls()
lf.head().collect()

Datetime,Close,High,Low,Open,Volume
"datetime[μs, EST]",f64,f64,f64,f64,i64
1980-12-11 19:00:00 EST,0.128348,0.128906,0.128348,0.128348,469033600
1980-12-14 19:00:00 EST,0.121652,0.12221,0.121652,0.12221,175884800
1980-12-15 19:00:00 EST,0.112723,0.113281,0.112723,0.113281,105728000
1980-12-16 19:00:00 EST,0.115513,0.116071,0.115513,0.115513,86441600
1980-12-17 19:00:00 EST,0.118862,0.11942,0.118862,0.118862,73449600


In [4]:
from torch.utils.data import DataLoader

dataset_kwargs = {'lf': lf,
                 'non_temporal': data_entry.non_temporal,
                 'temporal': data_entry.temporal,
                 'target': 'Close',
                 'look_back_len': 60,
                 'forecast_len': 5}

rnn_dataset = RNNDataset(**dataset_kwargs)

In [5]:
train_dataset, test_dataset = train_test_split(rnn_dataset, 0.01)
print(len(rnn_dataset), len(train_dataset), len(test_dataset))

#train_loader = DataLoader(train_dataset, batch_size=128, shuffle=False, drop_last=False)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False, drop_last=False)
data_loader = DataLoader(rnn_dataset, batch_size=128, shuffle=False, drop_last=False)

10945 10836 109


In [6]:
from tradeforecast import LSTM
from torch import nn, optim

lstm_kwargs = {'input_size': len(rnn_dataset.features),
              'hidden_size': 32,
              'n_LSTM': 2,
              'fc_out_size':[],
              'output_size': rnn_dataset.forecast_len,
              'dropout': 0}

lstm_model = LSTM(**lstm_kwargs)
print(f"The device is set to ---> '{lstm_model.device}'")

The device is set to ---> 'cuda'


In [7]:
lstm_model.train_model(nn.MSELoss, optim.SGD, 1500, data_loader, 0.0005)

Epoch: [1/1500]; Loss: 7.482814
Epoch: [2/1500]; Loss: 7.379873
Epoch: [3/1500]; Loss: 7.285176
Epoch: [4/1500]; Loss: 7.195868
Epoch: [5/1500]; Loss: 7.109045
Epoch: [6/1500]; Loss: 7.023448
Epoch: [7/1500]; Loss: 6.926224
Epoch: [8/1500]; Loss: 6.843916
Epoch: [9/1500]; Loss: 6.761056
Epoch: [10/1500]; Loss: 6.676913
Epoch: [11/1500]; Loss: 6.590847
Epoch: [12/1500]; Loss: 6.502467
Epoch: [13/1500]; Loss: 6.411311
Epoch: [14/1500]; Loss: 6.316920
Epoch: [15/1500]; Loss: 6.218882
Epoch: [16/1500]; Loss: 6.116842
Epoch: [17/1500]; Loss: 6.010485
Epoch: [18/1500]; Loss: 5.899544
Epoch: [19/1500]; Loss: 5.783802
Epoch: [20/1500]; Loss: 5.663101
Epoch: [21/1500]; Loss: 5.537357
Epoch: [22/1500]; Loss: 5.406547
Epoch: [23/1500]; Loss: 5.270703
Epoch: [24/1500]; Loss: 5.129882
Epoch: [25/1500]; Loss: 4.984154
Epoch: [26/1500]; Loss: 4.833589
Epoch: [27/1500]; Loss: 4.678275
Epoch: [28/1500]; Loss: 4.518337
Epoch: [29/1500]; Loss: 4.353978
Epoch: [30/1500]; Loss: 4.185508
Epoch: [31/1500]; L

In [8]:
y, y_preds = lstm_model.test_model(test_loader)
y, y_preds

(tensor([[ 9.1452,  9.1912,  9.2766,  9.2919,  9.3648],
         [ 9.1912,  9.2766,  9.2919,  9.3648,  9.2981],
         [ 9.2766,  9.2919,  9.3648,  9.2981,  9.4137],
         [ 9.2919,  9.3648,  9.2981,  9.4137,  9.2329],
         [ 9.3648,  9.2981,  9.4137,  9.2329,  9.9056],
         [ 9.2981,  9.4137,  9.2329,  9.9056, 10.1894],
         [ 9.4137,  9.2329,  9.9056, 10.1894, 10.2455],
         [ 9.2329,  9.9056, 10.1894, 10.2455, 10.1616],
         [ 9.9056, 10.1894, 10.2455, 10.1616, 10.3620],
         [10.1894, 10.2455, 10.1616, 10.3620, 10.2479],
         [10.2455, 10.1616, 10.3620, 10.2479, 10.0269],
         [10.1616, 10.3620, 10.2479, 10.0269,  9.9219],
         [10.3620, 10.2479, 10.0269,  9.9219,  9.9531],
         [10.2479, 10.0269,  9.9219,  9.9531,  9.9977],
         [10.0269,  9.9219,  9.9531,  9.9977, 10.1981],
         [ 9.9219,  9.9531,  9.9977, 10.1981, 10.2388],
         [ 9.9531,  9.9977, 10.1981, 10.2388, 10.0720],
         [ 9.9977, 10.1981, 10.2388, 10.0720, 10

In [9]:
from sklearn.metrics import mean_absolute_percentage_error
from torch import Tensor

y: Tensor = y; y_preds: Tensor = y_preds
y_arr, y_preds_arr = y.cpu().numpy(), y_preds.cpu().numpy()
y_arr.shape, y_preds_arr.shape

((109, 5), (109, 5))

In [10]:
rnn_dataset.inverse_transform(y_arr), rnn_dataset.inverse_transform(y_preds_arr)

(array([[191.29   , 192.25002, 194.03   , 194.35   , 195.87   ],
        [192.25002, 194.03   , 194.35   , 195.87   , 194.48   ],
        [194.03   , 194.35   , 195.87   , 194.48   , 196.89   ],
        [194.35   , 195.87   , 194.48   , 196.89   , 193.12   ],
        [195.87   , 194.48   , 196.89   , 193.12   , 207.15   ],
        [194.48   , 196.89   , 193.12   , 207.15   , 213.07   ],
        [196.89   , 193.12   , 207.15   , 213.07   , 214.24002],
        [193.12   , 207.15   , 213.07   , 214.24002, 212.49002],
        [207.15   , 213.07   , 214.24002, 212.49002, 216.67001],
        [213.07   , 214.24002, 212.49002, 216.67001, 214.29001],
        [214.24002, 212.49002, 216.67001, 214.29001, 209.68   ],
        [212.49002, 216.67001, 214.29001, 209.68   , 207.49   ],
        [216.67001, 214.29001, 209.68   , 207.49   , 208.14   ],
        [214.29001, 209.68   , 207.49   , 208.14   , 209.07002],
        [209.68   , 207.49   , 208.14   , 209.07002, 213.25   ],
        [207.49   , 208.1

In [11]:
mean_absolute_percentage_error(y_arr, y_preds_arr)

np.float64(0.08834478479178429)

In [12]:
model_fname = lstm_model.save_model_state(ticker_interval='AAPL_1d_base')
model_fname

'AAPL_1d_base_LSTM_13605_5-cuda.pth'